In [1]:
from qiskit import IBMQ
import sklearn.datasets as skd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.providers.ibmq import least_busy
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from sklearn.metrics import classification_report
from sklearn.preprocessing import normalize
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute
# from imblearn.under_sampling import RandomUnderSampler
import math

In [2]:
ds = pd.read_csv('Breast_cancer_data.csv')
ds.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


In [3]:
#defining dependent and independent variables
X = ds.drop('diagnosis', axis=1)
y = ds['diagnosis']

In [4]:
X.shape

(569, 5)

In [5]:
# X = X.values
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [6]:
# Rotator
from typing import Callable, List, Optional, Tuple, Union
from math import atan2
def rotate_matrix_nd(bs,degrees):
    ang = np.deg2rad(degrees)
    dim = np.shape(bs)[0]
    Rot_mat = np.eye(dim)
    for k in range(0,dim-2):
        for l in range(dim-1,k,-1):
            a = atan2(bs[l,k],bs[l-1,k])
            R = np.eye(dim)
            R_temp = np.array([[np.cos(a),np.sin(a)],[-1*np.sin(a),np.cos(a)]])
            tmp = l - 1
            R[tmp:tmp+R_temp.shape[0],tmp:tmp+R_temp.shape[0]] = R_temp
            bs=np.matmul(R,bs)
            Rot_mat = np.matmul(R,Rot_mat)
    R = np.eye(dim)
    R[dim-2:dim,dim-2:dim] = np.array([[np.cos(ang),-1*np.sin(ang)],[np.sin(ang),np.cos(ang)]])
    Rot_mat = np.matmul(np.linalg.inv(Rot_mat),np.matmul(R,Rot_mat))
    return Rot_mat

In [7]:
dim = 5
v = np.logical_or(np.eye(dim),np.fliplr(np.eye(dim))).astype(int)
v = v[:,:-2]
# v = np.eye(dim)[:,:-2]
# v[-2][0] = 1
# v[-1][1] = 1


# print(rot)

In [17]:
random_seed = 42
feature_dim = 5
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=3)
backend =  BasicAer.get_backend('qasm_simulator')#'statevector_simulator''qasm_simulator'
quantum_instance = QuantumInstance(backend, seed_simulator=random_seed, seed_transpiler=random_seed)

kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
qsvc = QSVC(quantum_kernel=kernel)



for angul in [91.005]:#range(-16,-8,1):
    file = open("result_"+str(angul)+".txt", "w")
    print("Voy en "+str(angul))
    rot = rotate_matrix_nd(v,-angul)
    XX = np.empty((0,dim))
    for point in X:
        tmp_point = np.matmul(rot,np.atleast_2d(point).T)
        tmp_point = np.atleast_2d(tmp_point).T
        XX = np.append(XX, tmp_point,axis=0)
    sample_train, sample_test, label_train, label_test = train_test_split(XX, y, test_size=0.4,  shuffle=True, random_state=42)
    
    
    qsvc.fit(sample_train, label_train)
    
    print('\n---------------------------- Angle:'+str(angul)+'----------------------------\n', file=file)
    print('\n----------------------------'+'TEST'+'----------------------------\n', file=file)
    print(classification_report(label_test , qsvc.predict(sample_test)), file=file)
    print('\n----------------------------'+'TRAIN'+'----------------------------\n', file=file)
    print(classification_report(label_train , qsvc.predict(sample_train)), file=file)
    file.close()


Voy en 91.005


22.0